In [1]:
import numpy as np
import pandas as pd
import folium
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import json
%matplotlib inline


In [2]:
# find the canton id in the json file
with open('ch-cantons.topojson.json') as data_file:    
    data = json.load(data_file)
canton_id=[]
for i in range(len(data["objects"]['cantons']['geometries'])):
    canton_id.append(data["objects"]['cantons']['geometries'][i]['id'])

In [3]:
#import the grant data
data=pd.read_csv('P3_GrantExport.csv',sep=';')

In [5]:
#drop every column except university institution and approved amount
data=data.drop(data.columns[[0,1,2,3,4,5,8,9,10,11,12,14]], axis=1)

#drop the data for which we dont have university because the description of the data
#mentions that if this field is empty the research is notconducted in a swiss institute
double_NaN_idx = data[pd.isnull(data['University'])].index.values 
data = data.drop(data.index[[double_NaN_idx]],axis=0)
data = data.reset_index('level_0')
data = data.drop('index', axis=1)

#drop the data for which we have no amount
no_amount_idx = data[np.equal(data['Approved Amount'], 'data not included in P3')].index.values
data = data.drop(data.index[[no_amount_idx]],axis=0)
data = data.reset_index('level_0')
data = data.drop('index', axis=1)

# transform the approved amount in float
data['Approved Amount']=data['Approved Amount'].astype(float)

#we also have to remove the data that endswith or NA (the line university doesn't contain information)
to_dropNA=data[data.University.str.endswith('NA')==True].index.values
#create the dataframe with the grants for which we have the university info
data=data.drop(data.index[[to_dropNA]],axis=0)
# reset the index
data.reset_index(inplace=True)
#remove the old index column
data=data.drop(data.columns[0], axis=1)

In [8]:
#add a column for the canton
data["Canton"] = ""
data

,Institution,University,Approved Amount,Canton
0,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,41022.0,
1,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",79732.0,
2,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,52627.0,
3,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",120042.0,
4,"Séminaire de politique économique, d'économie ...",Université de Fribourg - FR,53009.0,
5,Institut für ökumenische Studien Université de...,Université de Fribourg - FR,25403.0,
6,Ostasiatisches Seminar Universität Zürich,Universität Zürich - ZH,47100.0,
7,NaN,Université de Lausanne - LA,25814.0,
8,Laboratoire de Didactique et Epistémologie des...,Université de Genève - GE,360000.0,
9,Klinische Psychologie und Psychotherapie Insti...,Université de Fribourg - FR,153886.0,


In [ ]:
# make the good data

tiret=data.University.str.find('-').astype(int)
#for each line
canton=[]
for i in range(len(data_uni)):
    # keep everything that is after that
    canton=data.University[[i]].str[tiret[i]+2:]
    data.Canton[[i]]=canton


/Users/lugrinclaire/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
data=data_uni

In [9]:
data.to_pickle('data_incomplete_cantons')
data

,Institution,University,Approved Amount,Canton
0,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,41022.0,
1,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,52627.0,
2,"Séminaire de politique économique, d'économie ...",Université de Fribourg - FR,53009.0,
3,Institut für ökumenische Studien Université de...,Université de Fribourg - FR,25403.0,
4,Ostasiatisches Seminar Universität Zürich,Universität Zürich - ZH,47100.0,
5,NaN,Université de Lausanne - LA,25814.0,
6,Laboratoire de Didactique et Epistémologie des...,Université de Genève - GE,360000.0,
7,Klinische Psychologie und Psychotherapie Insti...,Université de Fribourg - FR,153886.0,
8,Département de Sociologie Faculté des Sciences...,Université de Genève - GE,116991.0,
9,Institut für Volkswirtschaft WWZ / FGS Univers...,Universität Basel - BS,112664.0,


In [ ]:
#print the list of identified cantons
print(data.Canton.unique())

#print the list of wanted cantons
print(canton_id)

In [ ]:
# replace the LA by VD
data.Canton[data.Canton=='LA']='VD'
#replace ETHZ by ZH
data.Canton[data.Canton=='ETHZ']='ZH'
#replace EPFL by LA
data.Canton[data.Canton=='EPFL']='VD'

In [ ]:
#find the data that already have a good canton
data_sorted=[]
for i in range(len(data)):
    if data.Canton[i] in canton_id:
        data_sorted.append(data[i])
        



In [ ]:
#treat the NPO data: take the institution and find their canton

In [ ]:
# find the cantons for all the remaining universities 

# Make the fake data

In [22]:
#find where is the -
tiret=data_uni.University.str.find('-').astype(int)
#for each line
canton=[]
for i in range(len(data_uni)):
    # keep everything that is after that
    canton=data_uni.University[[i]].str[tiret[i]+2:]
    
    #########ATTTENTIOOOOOOOOON A changer!!!
    if str(canton.values[0]) in canton_id:
        #write the canton in the canton column
        data_uni.Canton[[i]]=canton
    else:
        data_uni.Canton[[i]]='ZH'
data_uni.head()


# we have to replce LA with VD and probably some other things

/Users/lugrinclaire/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/lugrinclaire/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Institution,University,Approved Amount,Canton
0,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,41022.0,GE
1,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,52627.0,BS
2,"Séminaire de politique économique, d'économie ...",Université de Fribourg - FR,53009.0,FR
3,Institut für ökumenische Studien Université de...,Université de Fribourg - FR,25403.0,FR
4,Ostasiatisches Seminar Universität Zürich,Universität Zürich - ZH,47100.0,ZH


# analyse the fake data

In [24]:
data_uni.to_pickle('data_uni_fake')

In [25]:
data_uni.Canton.unique()

array(['GE', 'BS', 'FR', 'ZH', 'BE', 'NE', 'SG', 'LU'], dtype=object)

In [61]:
#sum all the data per canton
data_grouped=pd.DataFrame([])
money=[]
for i in canton_id:
    money.append(np.sum(data_uni['Approved Amount'][data_uni.Canton==i]))
data_grouped['Canton']=canton_id
data_grouped['Money']=money
data_grouped.set_index('Canton')

,Money
Canton,
ZH,6.841544e+09
BE,1.519373e+09
LU,3.998168e+07
UR,0.000000e+00
SZ,0.000000e+00
OW,0.000000e+00
NW,0.000000e+00
GL,0.000000e+00
ZG,0.000000e+00


In [87]:
#import the topojson map
topo_path = r'ch-cantons.topojson.json'
swiss_map = folium.Map(location=[46.8, 8.4], zoom_start=8)

In [88]:
#To use a Dataframe (in the DF, I used the id as the _Cantons_ and I put random values inside the _Values_ field), you need to do this:
swiss_map.choropleth(geo_path=topo_path, 
                     data=data_grouped,
                     columns=['Canton', 'Money'],
                     key_on='feature.id',
                     topojson='objects.cantons',
                     fill_color='BuPu',
                     legend_name = 'Grant money',
                     fill_opacity=0.7,
                     line_opacity=0.2
                    )
swiss_map
#In you DataFrame, you must be sure that *all the id present in the topojson are also present in the column of the DF*. If it's not the case, just add the missing ids..

/Users/lugrinclaire/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: 'threshold_scale' default behavior has changed. Now you get a linear scale between the 'min' and the 'max' of your data. To get former behavior, use folium.utilities.split_six.
